# Filters and combines existing clusters based on tools used, start and end dates, and organization
## Rebuilds all clusters (will need to run daily)

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ContactToolAssociation__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"sf2021shitOPmlIiFMLnrudgC6oSX0WV1T",   
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [2]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## Pull data from SF - this will be a post SF filter

In [3]:
##########################################################################################
## 1 - Pull all tool_usage_cluster__c information
##     i) ID
##     ii) Ending_Date__c
##     iii) Starting_Date__c
##     iv) Organization__c
##     v) Semester__c
## 2 - Pull all ContactToolClusterAssociation__c information
##     i) Tool_Usage_Cluster__c
##     ii) Contact__c
## 3 - Pull all ToolToolClusterAssociation__c information
##     i) Tool_Usage_Cluster__c
##     ii) Tool__c
## 4 - determine which tool_usage_cluster__c are suspicious 
##     i) first, group tool_usage_cluster__c by tool__c used
##     ii) check starting and ending dates - if they overlap, make a note of the tool_usage_clusters
## 5 - combining tool_usage_cluster__c 
##     i) use one of the existing Ids, mark the others for deletion
##     ii) when combining, pull the relevant Tool__c information and Contact__c information
##
##########################################################################################

In [4]:
db_1 = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [5]:
tool_cluster_df = db_1.query_data('Select ID, Ending_Date__c, Starting_Date__c, Organization__c,Semester__c \
                        from tool_usage_cluster__c')

[Success] Bulk job creation successful. Job ID = 7505w00000THO3dAAH
{"id":"7505w00000THO3dAAH","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-13T01:40:55.000+0000","systemModstamp":"2021-01-13T01:40:56.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000THO3dAAH","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-13T01:40:55.000+0000","systemModstamp":"2021-01-13T01:40:57.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":3936,"retries":0,"totalProcessingTime":624}
[Success] Bulk job completed successfully.


In [7]:
display(tool_cluster_df.head(5))

,Ending_Date__c,Id,Organization__c,Semester__c,Starting_Date__c
0,2013-09-20,a0w5w000009Q7CqAAK,a0r5w00000V42cCAAR,Fall,2013-09-13
1,2013-09-20,a0w5w000009Q7CrAAK,a0r5w00000V42cCAAR,Fall,2013-09-15
2,2013-09-20,a0w5w000009Q7CsAAK,a0r5w00000V42cCAAR,Fall,2013-09-15
3,2013-09-17,a0w5w000009Q7CtAAK,a0r5w00000V42cCAAR,Fall,2013-09-17
4,2011-01-30,a0w5w000009Q7CuAAK,NaN,Fall,2010-09-05


In [66]:
contact_in_cluster_df = db_1.query_data('Select Id, Name, Tool_Usage_Cluster__c, Contact__c from ContactToolClusterAssociation__c')

[Success] Bulk job creation successful. Job ID = 7505w00000THO7VAAX
{"id":"7505w00000THO7VAAX","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-13T01:50:00.000+0000","systemModstamp":"2021-01-13T01:50:00.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000THO7VAAX","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-13T01:50:00.000+0000","systemModstamp":"2021-01-13T01:50:10.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":92372,"retries":0,"totalProcessingTime":5570}
[Success] Bulk job completed successfully.


In [67]:
display(contact_in_cluster_df.head(5))

,Contact__c,Id,Name,Tool_Usage_Cluster__c
0,0035w000034JtebAAC,a0x5w00000ZtjVpAAJ,ksmithca22_59,a0w5w000009Q7DnAAK
1,0035w000034I5Y6AAK,a0x5w00000ZtjVqAAJ,lhamill_2319,a0w5w000009Q8G8AAK
2,0035w000034JqhvAAC,a0x5w00000ZtjVrAAJ,bkredito_2319,a0w5w000009Q8G8AAK
3,0035w000034Jqh2AAC,a0x5w00000ZtjVsAAJ,jones.a.doug_2319,a0w5w000009Q8G8AAK
4,0035w000034JqhZAAS,a0x5w00000ZtjVtAAJ,rhmarsha_2319,a0w5w000009Q8G8AAK


In [90]:
tools_in_cluster_df = db_1.query_data('Select Id, Name, Tool_Usage_Cluster__c, Tool__c from ToolToolClusterAssociation__c')

[Success] Bulk job creation successful. Job ID = 7505w00000THO9gAAH
{"id":"7505w00000THO9gAAH","operation":"query","object":"ToolToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-13T01:58:05.000+0000","systemModstamp":"2021-01-13T01:58:05.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000THO9gAAH","operation":"query","object":"ToolToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-13T01:58:05.000+0000","systemModstamp":"2021-01-13T01:58:07.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":5993,"retries":0,"totalProcessingTime":550}
[Success] Bulk job completed successfully.


In [91]:
display(tools_in_cluster_df.head(5))

,Id,Name,Tool_Usage_Cluster__c,Tool__c
0,a0y5w00000MSFA2AAP,geneticalgo_0,a0w5w000009Q7CqAAK,a0s5w00000k5MXTAA2
1,a0y5w00000MSFA3AAP,geneticalgo_1,a0w5w000009Q7CrAAK,a0s5w00000k5MXTAA2
2,a0y5w00000MSFA4AAP,geneticalgo_2,a0w5w000009Q7CsAAK,a0s5w00000k5MXTAA2
3,a0y5w00000MSFA5AAP,geneticalgo_3,a0w5w000009Q7CtAAK,a0s5w00000k5MXTAA2
4,a0y5w00000MSFA6AAP,bmcsuite_4,a0w5w000009Q7CuAAK,a0s5w00000k5MRtAAM


### determining suspicious tool_usage_clusters__c

In [12]:
## overarching goal: determine the cluster IDs that need further investigation

# grouping the clusters by tools used (i.e., clusters by tool)
tools_dict = {j: [] for j in tools_in_cluster_df['Tool__c'].to_list()} #stores relevant indexes per org

for i,j in enumerate(tools_in_cluster_df['Tool__c'].to_list()):
    tools_dict[j].append(i)

unique_tools = list(tools_dict.keys())

In [13]:
from copy import deepcopy

In [14]:
## check start and end dates for overlaps
overlaps = ['init'] # del index 0 later

cluster_ids = tools_in_cluster_df['Tool_Usage_Cluster__c'].to_list()
tool_cluster_id_indexes = tool_cluster_df['Id'].to_list()

for i,ii in enumerate(unique_tools):
    for j in tools_dict[ii]: # this requires a sequential comparison
        ## tools dict contains the tools_in_cluster_df index
        temp_set = deepcopy(tools_dict[ii])
        del temp_set[temp_set.index(j)]
        
        # pull id
        current_index = tool_cluster_id_indexes.index(cluster_ids[j]) #integer index
        
        current_org = tool_cluster_df.iloc[current_index,2]
        current_sem = tool_cluster_df.iloc[current_index,3]
        current_start = datetime.datetime.strptime(tool_cluster_df.iloc[current_index,-1],'%Y-%m-%d')
        current_stop = datetime.datetime.strptime(tool_cluster_df.iloc[current_index,0],'%Y-%m-%d')
        
        # sequential compare
        for k in temp_set:
            temp_index = tool_cluster_id_indexes.index(cluster_ids[k]) #integer index
            
            temp_org = tool_cluster_df.iloc[temp_index,2]
            temp_sem = tool_cluster_df.iloc[temp_index,3]
            temp_start = datetime.datetime.strptime(tool_cluster_df.iloc[temp_index,-1],'%Y-%m-%d')
            temp_stop = datetime.datetime.strptime(tool_cluster_df.iloc[temp_index,0],'%Y-%m-%d')
        
            if temp_org == current_org and temp_sem == current_sem:
                # now check dates for overlap - if there is overlap, assume same cluster
                if temp_start > current_stop or temp_stop < current_start: #case 1 - () [], # case 2 - [] ()
                    # do nothing
                    filler = 0
                else:
                    # include the overlap as a tuple
                    overlaps.append([cluster_ids[j],cluster_ids[k]])


In [15]:
## filter the set of overlaps
if overlaps[0] == 'init':
    del overlaps[0]

for i,j in enumerate(overlaps):
    j.sort()
    overlaps[i] = j

overlaps.sort()

In [16]:
overlaps[:5]

[['a0w5w000009Q7CqAAK', 'a0w5w000009Q7CrAAK'],
 ['a0w5w000009Q7CqAAK', 'a0w5w000009Q7CrAAK'],
 ['a0w5w000009Q7CqAAK', 'a0w5w000009Q7CsAAK'],
 ['a0w5w000009Q7CqAAK', 'a0w5w000009Q7CsAAK'],
 ['a0w5w000009Q7CqAAK', 'a0w5w000009Q7CtAAK']]

In [17]:
import itertools

In [18]:
overlaps2 = list(k for k,_ in itertools.groupby(overlaps))

In [19]:
overlaps2[:9]

[['a0w5w000009Q7CqAAK', 'a0w5w000009Q7CrAAK'],
 ['a0w5w000009Q7CqAAK', 'a0w5w000009Q7CsAAK'],
 ['a0w5w000009Q7CqAAK', 'a0w5w000009Q7CtAAK'],
 ['a0w5w000009Q7CrAAK', 'a0w5w000009Q7CsAAK'],
 ['a0w5w000009Q7CrAAK', 'a0w5w000009Q7CtAAK'],
 ['a0w5w000009Q7CsAAK', 'a0w5w000009Q7CtAAK'],
 ['a0w5w000009Q7CwAAK', 'a0w5w000009Q7lYAAS'],
 ['a0w5w000009Q7CxAAK', 'a0w5w000009Q80HAAS'],
 ['a0w5w000009Q7CzAAK', 'a0w5w000009Q7D0AAK']]

In [20]:
print(len(overlaps),len(overlaps2))

2620 1093


#### now that we have the set of overlapping clusters, we need to specify a main one

In [21]:
## main cluster characteristics
## Part A
## 1. combine overlaps together (i.e., three overlapping clusters combined into 1) 
## 2. decide the main cluster
## Part B
## 1. start and end dates realignment - i.e., should take the min of all the start dates and max of all the ends
## 2. combine contacts
## 3. create two DataFrames, one for upsert and the other for deletion

In [22]:
## Part A
# collapse overlaps2
overlaps3 = ['init']
main_clusters = []

running_temp_init = True

for i,j in enumerate(overlaps2):
    if running_temp_init == True:
        main_clusters.append(j[0])
        overlaps_holder = deepcopy(j)
        running_temp_init = False
    else:
        if j[0] in overlaps_holder and j[1] not in overlaps_holder:
            overlaps_holder.append(j[1])
        else: 
            if j[0] not in overlaps_holder and j[1] not in overlaps_holder: #neither is in overlaps_holder
                overlaps3.append(overlaps_holder)

                # recalc
                main_clusters.append(j[0])
                overlaps_holder = deepcopy(j)

# final iteration
overlaps3.append(overlaps_holder)

In [23]:
len(main_clusters)

427

In [24]:
if overlaps3[0] == 'init':
    del overlaps3[0]
    
print(len(overlaps3))

427


In [25]:
display(main_clusters[:5])
display(overlaps3[:5])

['a0w5w000009Q7CqAAK',
 'a0w5w000009Q7CwAAK',
 'a0w5w000009Q7CxAAK',
 'a0w5w000009Q7CzAAK',
 'a0w5w000009Q7D7AAK']

[['a0w5w000009Q7CqAAK',
  'a0w5w000009Q7CrAAK',
  'a0w5w000009Q7CsAAK',
  'a0w5w000009Q7CtAAK'],
 ['a0w5w000009Q7CwAAK', 'a0w5w000009Q7lYAAS'],
 ['a0w5w000009Q7CxAAK', 'a0w5w000009Q80HAAS'],
 ['a0w5w000009Q7CzAAK', 'a0w5w000009Q7D0AAK', 'a0w5w000009Q7D1AAK'],
 ['a0w5w000009Q7D7AAK', 'a0w5w000009Q7D8AAK']]

### Major step - building upsert and deletion dataframes
### Build 3 upserts and 3 deletions [corresponds to tool_usage_cluster__c, ContactToolClusterAssociation__c, ToolToolClusterAssociation__c]

In [26]:
## Part B
### build the upsert dataframe first
upsert_df = pd.DataFrame(columns=['Id','Ending_Date__c','Starting_Date__c']) #Id of the main cluster
upsert_df['Id'] = deepcopy(main_clusters)

In [27]:
for i,ii in enumerate(main_clusters):
    main_c_index = tool_cluster_df['Id'].to_list()
    c_start_date = tool_cluster_df.iloc[main_c_index.index(ii),-1]    
    c_end_date = tool_cluster_df.iloc[main_c_index.index(ii),0]
        
    for j,jj in enumerate(overlaps3[i]):
        j_start_date = tool_cluster_df.iloc[main_c_index.index(jj),-1]
        j_end_date = tool_cluster_df.iloc[main_c_index.index(jj),0]
        
        if j_start_date < c_start_date:
            c_start_date = deepcopy(j_start_date)
        
        if j_end_date > c_end_date:
            c_end_date = deepcopy(j_end_date)
        
    upsert_df.iloc[i,1:] = [c_end_date,c_start_date]

In [28]:
display(upsert_df.head(5))
display(upsert_df.tail(5))

,Id,Ending_Date__c,Starting_Date__c
0,a0w5w000009Q7CqAAK,2013-09-20,2013-09-13
1,a0w5w000009Q7CwAAK,2019-11-28,2019-11-04
2,a0w5w000009Q7CxAAK,2017-12-16,2017-10-03
3,a0w5w000009Q7CzAAK,2018-04-19,2018-02-05
4,a0w5w000009Q7D7AAK,2006-12-14,2006-09-23


,Id,Ending_Date__c,Starting_Date__c
422,a0w5w00000A8NYUAA3,2020-12-09,2020-11-07
423,a0w5w00000A8NYYAA3,2020-12-05,2020-11-27
424,a0w5w00000A8NYgAAN,2020-11-01,2020-10-13
425,a0w5w00000A8NYvAAN,2020-11-13,2020-11-08
426,a0w5w00000A8NYzAAN,2020-12-05,2020-11-22


In [29]:
### buliding the delete dataframe - should be all Ids in overlaps that are not main clusters
deletion_df = pd.DataFrame(columns=['Id']) #Id of the main cluster

deletion_ids = []

for i,ii in enumerate(main_clusters):
    for j in overlaps3[i][1:]:
        deletion_ids.append(j)
deletion_df['Id'] = deletion_ids

In [30]:
display(deletion_df.head(3))

,Id
0,a0w5w000009Q7CrAAK
1,a0w5w000009Q7CsAAK
2,a0w5w000009Q7CtAAK


In [31]:
## summary up to this point
## upsert_df db.send_data(upsert_df)
## db.object_id = 'tool_usage_cluster__c'
## db.external_id = 'Id'

## deletion_df db.delete_data(deletion_df)
## db.object_id = 'tool_usage_cluster__c'
## db.external_id = 'Id'

In [68]:
## updating contact and cluster mapping
display(contact_in_cluster_df.head(2))
print(contact_in_cluster_df.shape)

,Contact__c,Id,Name,Tool_Usage_Cluster__c
0,0035w000034JtebAAC,a0x5w00000ZtjVpAAJ,ksmithca22_59,a0w5w000009Q7DnAAK
1,0035w000034I5Y6AAK,a0x5w00000ZtjVqAAJ,lhamill_2319,a0w5w000009Q8G8AAK


(92372, 4)


In [69]:
# NOTE: there are contacts that belong to non-overlapping clusters
contact_usage_clusters = contact_in_cluster_df['Tool_Usage_Cluster__c'].to_list()
contacts = contact_in_cluster_df['Contact__c'].to_list()
change_indexes = []

## check if current cluster belongs to overlaps3 - first need to convert overlaps3 into a direct list
overlaps4 = list(itertools.chain.from_iterable(overlaps3))

In [70]:
print(len(overlaps4))

1232


In [71]:
## populate the contacts that need changing
for i,j in enumerate(contact_usage_clusters):
    if j in overlaps4:
        change_indexes.append(i)

In [72]:
print(len(change_indexes))

37321


In [73]:
## search for each index that needs changing
for i in change_indexes:
    if contact_usage_clusters[i] not in main_clusters:
        #print('testouter')
        
        # need to find its main_cluster
        found_main = False
        main_counter = 0
        while found_main == False:
            if contact_usage_clusters[i] in overlaps3[main_counter]:
                #print('test')
                found_main = True
                contact_usage_clusters[i] = overlaps3[main_counter][0]
            else:
                main_counter += 1

In [74]:
contact_in_cluster_df['Tool_Usage_Cluster__c'] = contact_usage_clusters

In [75]:
## remove duplicates from contact_in_cluster_df
contact_in_cluster_df = contact_in_cluster_df.drop_duplicates()
display(contact_in_cluster_df.head(2))
print(contact_in_cluster_df.shape)

,Contact__c,Id,Name,Tool_Usage_Cluster__c
0,0035w000034JtebAAC,a0x5w00000ZtjVpAAJ,ksmithca22_59,a0w5w000009Q7DnAAK
1,0035w000034I5Y6AAK,a0x5w00000ZtjVqAAJ,lhamill_2319,a0w5w000009Q8G8AAK


(92372, 4)


In [92]:
## tools - take max, i.e. set all constituent clusters of a main cluster to the main cluster
## then remove duplicates
display(tools_in_cluster_df.head(2))

,Id,Name,Tool_Usage_Cluster__c,Tool__c
0,a0y5w00000MSFA2AAP,geneticalgo_0,a0w5w000009Q7CqAAK,a0s5w00000k5MXTAA2
1,a0y5w00000MSFA3AAP,geneticalgo_1,a0w5w000009Q7CrAAK,a0s5w00000k5MXTAA2


In [93]:
tuc = tools_in_cluster_df['Tool_Usage_Cluster__c'].to_list()

# first find those cluster ids that are clustered
tool_cluster_ids = []
for i,j in enumerate(tuc):
    if j in overlaps4:
        tool_cluster_ids.append(i)

In [94]:
print(len(tool_cluster_ids))

1957


In [95]:
for i in tool_cluster_ids:
    if tuc[i] not in main_clusters: #then need to find its corresponding main cluster
        found_main = False
        main_counter = 0
        
        while found_main == False:
            if tuc[i] in overlaps3[main_counter]:
                #print('test')
                found_main = True
                tuc[i] = overlaps3[main_counter][0]
            else:
                main_counter += 1

In [96]:
tools_in_cluster_df['Tool_Usage_Cluster__c'] = tuc
display(tools_in_cluster_df.head(2))

,Id,Name,Tool_Usage_Cluster__c,Tool__c
0,a0y5w00000MSFA2AAP,geneticalgo_0,a0w5w000009Q7CqAAK,a0s5w00000k5MXTAA2
1,a0y5w00000MSFA3AAP,geneticalgo_1,a0w5w000009Q7CqAAK,a0s5w00000k5MXTAA2


## Send to SF

In [45]:
db_upsert_del = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [46]:
db_upsert_del.object_id = 'tool_usage_cluster__c'
db_upsert_del.external_id = 'Id'

In [47]:
db_upsert_del.send_data(upsert_df)

[Success] Bulk job creation successful. Job ID = 7505w00000THO3KAAX
hello
[Success] CSV upload successful. Job ID = 7505w00000THO3KAAX
[Success] Closing job successful. Job ID = 7505w00000THO3KAAX


In [48]:
db_upsert_del.check_bulk_status()

{'id': '7505w00000THO3KAAX',
 'operation': 'upsert',
 'object': 'tool_usage_cluster__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T01:44:15.000+0000',
 'systemModstamp': '2021-01-13T01:44:15.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [49]:
from pprint import pprint
pprint(db_upsert_del.check_bulk_failed_results())

''


#### upload contact_in_cluster_df

In [79]:
db_contact_upsert = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [80]:
## need to delete first, then upload
contact_in_cluster_df_del = pd.DataFrame(contact_in_cluster_df['Id'])
contact_in_cluster_df_up = contact_in_cluster_df.drop(columns='Id')

In [81]:
display(contact_in_cluster_df_del.head(2))
display(contact_in_cluster_df_up.head(2))

,Id
0,a0x5w00000ZtjVpAAJ
1,a0x5w00000ZtjVqAAJ


,Contact__c,Name,Tool_Usage_Cluster__c
0,0035w000034JtebAAC,ksmithca22_59,a0w5w000009Q7DnAAK
1,0035w000034I5Y6AAK,lhamill_2319,a0w5w000009Q8G8AAK


In [82]:
db_contact_upsert.object_id = 'ContactToolClusterAssociation__c'
db_contact_upsert.external_id = 'Id'

In [83]:
db_contact_upsert.delete_data(contact_in_cluster_df_del)

[Success] Bulk job creation successful. Job ID = 7505w00000THO8OAAX
hello
[Success] CSV upload successful. Job ID = 7505w00000THO8OAAX
[Success] Closing job successful. Job ID = 7505w00000THO8OAAX


In [102]:
db_contact_upsert.check_bulk_status()

{'id': '7505w00000THO8OAAX',
 'operation': 'delete',
 'object': 'ContactToolClusterAssociation__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T01:54:25.000+0000',
 'systemModstamp': '2021-01-13T02:00:52.000+0000',
 'state': 'JobComplete',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 92372,
 'numberRecordsFailed': 37,
 'retries': 0,
 'totalProcessingTime': 1770501,
 'apiActiveProcessingTime': 1688520,
 'apexProcessingTime': 5925037}

In [108]:
pprint(db_contact_upsert.check_bulk_failed_results())

''


In [ ]:
## now send data

In [104]:
db_contact_upsert.object_id = 'ContactToolClusterAssociation__c'
db_contact_upsert.external_id = 'Name'

In [105]:
db_contact_upsert.send_data(contact_in_cluster_df_up)

[Success] Bulk job creation successful. Job ID = 7505w00000THOCzAAP
hello
[Success] CSV upload successful. Job ID = 7505w00000THOCzAAP
[Success] Closing job successful. Job ID = 7505w00000THOCzAAP


In [110]:
db_contact_upsert.check_bulk_status()

{'id': '7505w00000THOCzAAP',
 'operation': 'upsert',
 'object': 'ContactToolClusterAssociation__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T02:02:09.000+0000',
 'systemModstamp': '2021-01-13T02:07:04.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'Name',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 92372,
 'numberRecordsFailed': 1803,
 'retries': 0,
 'totalProcessingTime': 1288684,
 'apiActiveProcessingTime': 1206454,
 'apexProcessingTime': 5929997}

In [111]:
pprint(db_contact_upsert.check_bulk_failed_results())

('"sf__Id","sf__Error",Contact__c,Name,Tool_Usage_Cluster__c\n'
 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '184 records: '
 '0035w000034K4hWAAS,0035w000034K4hXAAS,0035w000034K4hUAAS,0035w000034K4hVAAS,0035w000034Jl8LAAS,0035w000034K4hSAAS,0035w000034JLc5AAG,0035w000034JLzkAAG,0035w000034JNfUAAW,0035w000034K4hOAAS, '
 '... (174 '
 'more):--","0035w000034K4iKAAS","jbarsima_3445","a0w5w000009Q8YIAA0"\n'
 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '184 records: '
 '0035w000034K4hWAAS,0035w000034K4hXAAS,0035w000034K4hUAAS,0035w000034K4hVAAS,0035w000034Jl8LAAS,0035w000034K4hSAAS,0035w000034JLc5AAG,0035w000034JLzkAAG,0035w000034JNfUAAW,0035w000034K4hOAAS, '
 '... (174 '
 'more):--","0035w000034K4qNAAS","myers283_3445","a0w5w000009Q8YIAA0"\n'
 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '184 records: '
 '0035w000034K4hWAAS,0035w000034K4hXAAS,0035w000034K4hUAAS,0035w000034K4hVAAS,0035w000

 '... (166 more):--","0035w000031Vx1IAAS","mmkfc_538","a0w5w000009Q7nIAAS"\n'
 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '176 records: '
 '0035w000034JsVeAAK,0035w000031VxGZAA0,0035w000034IryGAAS,0035w000031VxGaAAK,0035w000034IrxdAAC,0035w000034JH8lAAG,0035w000031VxFtAAK,0035w000031VxH4AAK,0035w000034Jl8DAAS,0035w000031VxGXAA0, '
 '... (166 more):--","0035w000034JHc2AAG","mohamed_540","a0w5w000009Q7nIAAS"\n'
 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '176 records: '
 '0035w000034JsVeAAK,0035w000031VxGZAA0,0035w000034IryGAAS,0035w000031VxGaAAK,0035w000034IrxdAAC,0035w000034JH8lAAG,0035w000031VxFtAAK,0035w000031VxH4AAK,0035w000034Jl8DAAS,0035w000031VxGXAA0, '
 '... (166 more):--","0035w000034JIwOAAW","mkoh_540","a0w5w000009Q7nIAAS"\n'
 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '176 records: '
 '0035w000034JsVeAAK,0035w000031VxGZAA0,0035w000034IryGAAS,0035w000031VxGaAAK,0035w0000

 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '159 records: '
 '0035w000031VuAYAA0,0035w000031VuAXAA0,0035w000031VvKfAAK,0035w000034JKFyAAO,0035w000031VtpmAAC,0035w000031VtiFAAS,0035w000031VtpnAAC,0035w000034JM3BAAW,0035w000034I2YeAAK,0035w000031VtplAAC, '
 '... (149 '
 'more):--","0035w000034JMMCAA4","arun2987_3489","a0w5w000009Q8Z0AAK"\n'
 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '159 records: '
 '0035w000031VuAYAA0,0035w000031VuAXAA0,0035w000031VvKfAAK,0035w000034JKFyAAO,0035w000031VtpmAAC,0035w000031VtiFAAS,0035w000031VtpnAAC,0035w000034JM3BAAW,0035w000034I2YeAAK,0035w000031VtplAAC, '
 '... (149 more):--","0035w000034JLRSAA4","spalit_3489","a0w5w000009Q8Z0AAK"\n'
 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '159 records: '
 '0035w000031VuAYAA0,0035w000031VuAXAA0,0035w000031VvKfAAK,0035w000034JKFyAAO,0035w000031VtpmAAC,0035w000031VtiFAAS,0035w000031VtpnAAC,0035w000034JM3BAAW,00

 '177 records: '
 '0035w000034JHayAAG,0035w000034ISC0AAO,0035w000034IXdjAAG,0035w000034IX2uAAG,0035w000034IXdgAAG,0035w000034IXdhAAG,0035w000034IPlvAAG,0035w000034JusSAAS,0035w000034IsjJAAS,0035w000034JwGCAA0, '
 '... (167 more):--","0035w000034JExuAAG","meister_1155","a0w5w000009Q7xLAAS"\n'
 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '177 records: '
 '0035w000034JHayAAG,0035w000034ISC0AAO,0035w000034IXdjAAG,0035w000034IX2uAAG,0035w000034IXdgAAG,0035w000034IXdhAAG,0035w000034IPlvAAG,0035w000034JusSAAS,0035w000034IsjJAAS,0035w000034JwGCAA0, '
 '... (167 '
 'more):--","0035w000034JIMvAAO","busychic2005_1155","a0w5w000009Q7xLAAS"\n'
 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '177 records: '
 '0035w000034JHayAAG,0035w000034ISC0AAO,0035w000034IXdjAAG,0035w000034IX2uAAG,0035w000034IXdgAAG,0035w000034IXdhAAG,0035w000034IPlvAAG,0035w000034JusSAAS,0035w000034IsjJAAS,0035w000034JwGCAA0, '
 '... (167 more):--","0035w000034JH

 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '199 records: '
 '0035w000034JJo5AAG,0035w000031VucfAAC,0035w000034JKMdAAO,0035w000034JJJzAAO,0035w000034JJJxAAO,0035w000034JKMXAA4,0035w000034JJJtAAO,0035w000034JJJaAAO,0035w000034JJIwAAO,0035w000034JJK7AAO, '
 '... (189 more):--","0035w000034JJRhAAO","akim_950","a0w5w000009Q7u3AAC"\n'
 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '199 records: '
 '0035w000034JJo5AAG,0035w000031VucfAAC,0035w000034JKMdAAO,0035w000034JJJzAAO,0035w000034JJJxAAO,0035w000034JKMXAA4,0035w000034JJJtAAO,0035w000034JJJaAAO,0035w000034JJIwAAO,0035w000034JJK7AAO, '
 '... (189 more):--","0035w000034JJIvAAO","skramer_950","a0w5w000009Q7u3AAC"\n'
 '"","UNABLE_TO_LOCK_ROW:unable to obtain exclusive access to this record or '
 '199 records: '
 '0035w000034JJo5AAG,0035w000031VucfAAC,0035w000034JKMdAAO,0035w000034JJJzAAO,0035w000034JJJxAAO,0035w000034JKMXAA4,0035w000034JJJtAAO,0035w000034JJJaAAO,0035w000034

#### tools_in_cluster_df

In [112]:
db_tool_upsert = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [114]:
## need to delete first, then upload
db_tool_upsert_del = pd.DataFrame(tools_in_cluster_df['Id'])
db_tool_upsert_up = tools_in_cluster_df.drop(columns='Id')

In [115]:
display(db_tool_upsert_del.head(2))
display(db_tool_upsert_up.head(2))

,Id
0,a0y5w00000MSFA2AAP
1,a0y5w00000MSFA3AAP


,Name,Tool_Usage_Cluster__c,Tool__c
0,geneticalgo_0,a0w5w000009Q7CqAAK,a0s5w00000k5MXTAA2
1,geneticalgo_1,a0w5w000009Q7CqAAK,a0s5w00000k5MXTAA2


In [116]:
db_tool_upsert.external_id = 'Id'
db_tool_upsert.object_id = 'ToolToolClusterAssociation__c'

In [117]:
db_tool_upsert.delete_data(db_tool_upsert_del)

[Success] Bulk job creation successful. Job ID = 7505w00000THOLNAA5
hello
[Success] CSV upload successful. Job ID = 7505w00000THOLNAA5
[Success] Closing job successful. Job ID = 7505w00000THOLNAA5


In [122]:
db_tool_upsert.check_bulk_status()

{'id': '7505w00000THOLNAA5',
 'operation': 'delete',
 'object': 'ToolToolClusterAssociation__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T02:12:54.000+0000',
 'systemModstamp': '2021-01-13T02:13:25.000+0000',
 'state': 'JobComplete',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 5993,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 29401,
 'apiActiveProcessingTime': 27181,
 'apexProcessingTime': 0}

In [123]:
pprint(db_tool_upsert.check_bulk_failed_results())

'"sf__Id","sf__Error",Id\n'


In [124]:
## now upload

In [125]:
db_tool_upsert.external_id = 'Name'
db_tool_upsert.object_id = 'ToolToolClusterAssociation__c'

In [126]:
db_tool_upsert.send_data(db_tool_upsert_up)

[Success] Bulk job creation successful. Job ID = 7505w00000THOMGAA5
hello
[Success] CSV upload successful. Job ID = 7505w00000THOMGAA5
[Success] Closing job successful. Job ID = 7505w00000THOMGAA5


In [133]:
db_tool_upsert.check_bulk_status()

{'id': '7505w00000THOMGAA5',
 'operation': 'upsert',
 'object': 'ToolToolClusterAssociation__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T02:15:17.000+0000',
 'systemModstamp': '2021-01-13T02:15:42.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'Name',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 5993,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 22895,
 'apiActiveProcessingTime': 20777,
 'apexProcessingTime': 0}

In [134]:
pprint(db_tool_upsert.check_bulk_failed_results())

'"sf__Id","sf__Error",Name,Tool_Usage_Cluster__c,Tool__c\n'


#### delete clustered tool cluster ids

In [135]:
db_del = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [137]:
db_del.object_id = 'tool_usage_cluster__c'
db_del.external_id = 'id'

In [138]:
db_del.delete_data(deletion_df)

[Success] Bulk job creation successful. Job ID = 7505w00000THOMuAAP
hello
[Success] CSV upload successful. Job ID = 7505w00000THOMuAAP
[Success] Closing job successful. Job ID = 7505w00000THOMuAAP


In [139]:
display(deletion_df.head(2))

,Id
0,a0w5w000009Q7CrAAK
1,a0w5w000009Q7CsAAK


In [144]:
db_del.check_bulk_status()

{'id': '7505w00000THOMuAAP',
 'operation': 'delete',
 'object': 'tool_usage_cluster__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-13T02:17:12.000+0000',
 'systemModstamp': '2021-01-13T02:17:54.000+0000',
 'state': 'JobComplete',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 805,
 'numberRecordsFailed': 51,
 'retries': 0,
 'totalProcessingTime': 39991,
 'apiActiveProcessingTime': 38379,
 'apexProcessingTime': 16107}

In [145]:
pprint(db_del.check_bulk_failed_results())

('"sf__Id","sf__Error",Id\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q7nVAAS"\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q7nWAAS"\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q7nXAAS"\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q7sAAAS"\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q8FNAA0"\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q87zAAC"\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q887AAC"\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q888AAC"\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q8GWAA0"\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q7tkAAC"\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q7tlAAC"\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q8ZgAAK"\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q831AAC"\n'
 '"","ENTITY_IS_DELETED:entity is deleted:--","a0w5w000009Q83

In [ ]:
### eventually, want to update DB2 with this information
